In [1]:
import brightway2 as bw
import bw2regional as bwr
import bw2_lcimpact
from copy import copy
import constructive_geometries
import os
from bw2regional.pandarus_remote import AlreadyExists

In [2]:
bw.projects.set_current("EXIOBASE regionalized case study")

In [3]:
bw.databases

Databases dictionary with 3 object(s):
	EXIOBASE 3.3.17 hybrid
	Oil seeds
	biosphere3

In [4]:
bw.methods

Methods dictionary with 0 objects

In [5]:
bwr.bw2regionalsetup('world')

In [6]:
bwr.geocollections['world']

{'filepath': '/Users/cmutel/Library/Application Support/Brightway3/EXIOBASE-regionalized-case-study.9666a926e52c1cc772cb07068bf407f2/regional/countries.gpkg',
 'field': 'isotwolettercode',
 'sha256': '654f8ee414416b3e2c2a2829cac2ea4a35c77cacb48d4ed1f58155f8dff17799',
 'kind': 'vector'}

We need to define the Rest-of-Worlds used in EXIOBASE. Happily, they are only a few of them:

* **WA**:	RoW Asia and Pacific
* **WL**:	RoW America
* **WE**:	RoW Europe
* **WF**:	RoW Africa - really unfortunate acronym, as this is the code for an actual location...
* **WM**:	RoW Middle East

We will label the EXIOBASE Rest-of-Worlds with a) a new `Geocollection`, and b) more suitable names.

In [7]:
exiobase_row_mapping = {
    "WA": "RoW Asia and Pacific",
    "WL": "RoW America",
    "WE": "RoW Europe",
    "WF": "RoW Africa",
    "WM": "RoW Middle East",
}

In [8]:
bwr.geocollections['exiobase-rows'] = {
    'filepath': os.path.join(os.getcwd(), "exiobase-rows.gpkg"),
    'field': 'name'
}

In [9]:
for activity in bw.Database("EXIOBASE 3.3.17 hybrid"):
    if activity['location'] in exiobase_row_mapping:
        activity['location'] = ('exiobase-rows', exiobase_row_mapping[activity['location']])
        activity.save()

In [10]:
for activity in bw.Database("Oil seeds"):
    if activity['location'] in exiobase_row_mapping:
        activity['location'] = ('exiobase-rows', exiobase_row_mapping[activity['location']])
        activity.save()

In [ ]:
bw.databases['biosphere3']['geocollections'] = []
bw.databases['EXIOBASE 3.3.17 hybrid']['geocollections'] = ['world', 'exiobase-rows']
bw.databases['Oil seeds']['geocollections'] = ['world', 'exiobase-rows']
bw.databases.flush()

In a separate noteboom, created GIS data for these "rest-of-world" locations.

<img src="images/exiobase-row.png">

In [11]:
bw2_lcimpact.import_global_lcimpact()

In [12]:
bw2_lcimpact.import_regionalized_lcimpact()

Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Creating intersection (world, watersheds-hh)
Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Creating intersection (world, watersheds-eq-sw-certain)
Merging topographical faces for geocollection world


...

Creating intersection (world, world-topo-watersheds-eq-sw-all)
Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Creating intersection (world, world-topo-particulate-matter)
Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Creating intersection (world, world-topo-ecoregions)


In [14]:
bwr.remote.intersection(
    "exiobase-rows", 
    "watersheds-eq-sw-certain", 
)
bwr.remote.intersection(
    "exiobase-rows", 
    "watersheds-eq-sw-all", 
)
bwr.remote.intersection(
    "exiobase-rows", 
    "particulate-matter", 
)
bwr.remote.intersection(
    "exiobase-rows", 
    "ecoregions", 
)
bwr.remote.intersection(
    "exiobase-rows", 
    "watersheds-hh", 
)

('exiobase-rows', 'watersheds-hh')

In [13]:
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-eq-sw-certain", 
    'exiobase-rows-watersheds-eq-sw-certain'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-eq-sw-all", 
    'exiobase-rows-watersheds-eq-sw-all'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "particulate-matter", 
    'exiobase-rows-particulate-matter'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "ecoregions", 
    'exiobase-rows-ecoregions'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-hh", 
    'exiobase-rows-watersheds-hh'
)

In [8]:
from bw2regional import create_empty_intersection
# create_empty_intersection(('world', 'exiobase-rows-ecoregions'))
# create_empty_intersection(('exiobase-rows', 'ecoregions'))
# bwr.remote.calculate_intersection('exiobase-rows', 'world-topo-ecoregions')

Brightway2 Intersection: exiobase-rows: ecoregions

In [19]:
sorted(bwr.intersections)

[('ecoregions', 'exiobase-rows'),
 ('ecoregions', 'exiobase-rows-ecoregions'),
 ('ecoregions', 'world'),
 ('ecoregions', 'world-topo-ecoregions'),
 ('exiobase-rows', 'ecoregions'),
 ('exiobase-rows', 'exiobase-rows-ecoregions'),
 ('exiobase-rows', 'exiobase-rows-particulate-matter'),
 ('exiobase-rows', 'exiobase-rows-watersheds-eq-sw-all'),
 ('exiobase-rows', 'exiobase-rows-watersheds-eq-sw-certain'),
 ('exiobase-rows', 'exiobase-rows-watersheds-hh'),
 ('exiobase-rows', 'particulate-matter'),
 ('exiobase-rows', 'watersheds-eq-sw-all'),
 ('exiobase-rows', 'watersheds-eq-sw-certain'),
 ('exiobase-rows', 'watersheds-hh'),
 ('exiobase-rows-ecoregions', 'ecoregions'),
 ('exiobase-rows-ecoregions', 'exiobase-rows'),
 ('exiobase-rows-particulate-matter', 'exiobase-rows'),
 ('exiobase-rows-particulate-matter', 'particulate-matter'),
 ('exiobase-rows-watersheds-eq-sw-all', 'exiobase-rows'),
 ('exiobase-rows-watersheds-eq-sw-all', 'watersheds-eq-sw-all'),
 ('exiobase-rows-watersheds-eq-sw-certai

In [ ]:
other_scales = [
    'world-topo-watersheds-eq-sw-all',
    'world-topo-watersheds-eq-sw-certain',
    'world-topo-watersheds-hh',
    'world-topo-particulate-matter',
    'world-topo-ecoregions',    
]

for gc in other_scales:
    bwr.remote.intersection('exiobase-rows', gc)

# Where are crops actually grown?

Although this data source is not perfect, [Ramankutty et al](http://www.earthstat.org/harvested-area-yield-175-crops/) provide data on all our crops (note that peanuts can also be called groundnuts, as they are here).

I downloaded this data, and created compressed GeoTIFFS with `gdal_translate`:

    gdal_translate input_filename output_filename -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=DEFLATE

We also needed to transform the production rasters from total production per raster cell, to intensity (i.e. production per area, but note that this is total area, not harvested area). There is a function in `bw2regional` to do this already:

    bwr.divide_by_area(source, destination)


In [15]:
rasters = (
    ("gdpweighted.tiff", "weighted-pop-density"),
    ("oilpalm_intensity.tiff", "oilpalm"),
    ("rapeseed_intensity.tiff", "rapeseed"),
    ("soybean_intensity.tiff", "soybean"),
    ("groundnut_intensity.tiff", "peanut"),
    ("sunflower_intensity.tiff", "sunflower"),
)

In [16]:
third_scales = [
    'exiobase-rows-ecoregions',
    'exiobase-rows-particulate-matter',
    'exiobase-rows-watersheds-eq-sw-all',
    'exiobase-rows-watersheds-eq-sw-certain',
    'exiobase-rows-watersheds-hh',
    'world-topo-ecoregions',
    'world-topo-particulate-matter',
    'world-topo-watersheds-eq-sw-all',
    'world-topo-watersheds-eq-sw-certain',
    'world-topo-watersheds-hh'
]

In [17]:
for x, y in rasters:
    bwr.geocollections[y] = {
        'filepath': os.path.abspath("data/" + x),
        'band': 1
    }

In [18]:
for fp, raster in rasters:
    for gc in third_scales:
        bwr.remote.rasterstats_as_xt(gc, raster, f"xt-{raster}-{gc}")